In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Attentionは効果的で重要なテクニック

# Attentionの仕組み
「注意機構」(Attention mechanism)は，seq2seqに必要な情報へ注意を向けさせる

## seq2seqの問題点
Encoderの出力するベクトルの長さが固定長だと，洋服がタンスからあふれだすように，いずれ限界が訪れる．

## Encoderの改良
LSTMレイヤの隠れ状態ベクトルを「すべて」利用することで，入力される文章の長さに応じた情報を出力させる  
全ての時刻の隠れ状態を返すか，最後の隠れ状態ベクトルだけを返すかは，KerasのRNN初期化時のreturn_sequencesなどで変更できる  
<br>
Encoderの出力するベクトルは入力された単語の数だけベクトルがあり，それぞれのベクトルは各単語に対応した情報を多く含む
<br>
吾輩 は 猫 で ある -> $hs$ 
<div style="text-align: center">a b c d (「吾輩」要素強め)<br>
e f g h<br>
i j k l<br>
m n o p<br>
q r s t<br>
</div>
<br>

ここまでのRNNは過去の情報しか見えないが，全体の情報をバランスよく含めたければ**双方向RNN**や**双方向LSTM**を使う．  
ここでは，これまで通り単方向のLSTMを利用する．

## Decoderの改良1
$hs$をDecoderで扱うとき，どの行がどの単語に対応しているかを学習させる．  
単語やフレーズの対応関係を表す情報は**アライメント**と呼ばれる．  
Attentionという技術は，アライメントのアイデアをseq2seqで自動で取り入れることに成功した．  
このように，時系列データの変換のため，翻訳先と翻訳元の対応情報に注意を向けさせる仕組みを**Attention**と呼ぶ  
<br>
Attentionの仕組みを取り入れたDecoderは，LSTMへhsの最終行を渡すのと同時に，LSTMの各時刻からの出力と$hs$を受け取って何らかの計算をしたのち，Affine層へ出力するレイヤを設ける．  
例えば，最初のLSTMが「I」を出力するとき，その層では$hs$から「吾輩」に対応するベクトルを選び出す操作を，その何らかの計算で行う．  
しかし，選び出すという操作は基本的に微分ができないので，誤差逆伝搬法が使えない．  
なので，$hs$のすべてを選び出し，各行に各単語の重要度(貢献度)となる重み$a$をかけることで選び出しを行う．  
その重みをつけた各単語のベクトルの総和を**コンテキストベクトル**$c$として算出する．  
例えば「吾輩」に相当するベクトルの重みが0.8などの大きな値であるとき，$hs$から算出されるコンテキストベクトルは「吾輩」のベクトルに近いベクトルになり，これがベクトルの選び出しに相当する操作で，微分も可能なものになっている．  

In [17]:
# コンテキストベクトルの算出
T, H = 5,4 # 時系列の長さ，単語の隠れ層ベクトルの長さ
hs = np.random.randn(T, H)
print("hs", hs.shape)
print(hs, '\n')

a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])
print("a", a.shape)
print(a, '\n')

ar = a.reshape(5, 1).repeat(H, axis=1) # 縦にして列方向へ展開，repeatいらないが，あとでrepeatの逆伝搬が必要なことを明示的にしたい．
print("ar", ar.shape) # (5, 4)
print(ar, '\n')

t = hs * ar
print("t", t.shape) # (5, 4)
print(t, '\n')

c = np.sum(t, axis=0) # ex.(X, Y, Z) のaxis=1で和を取ると，(X, Z)のように，id=1が消える
print("c", c.shape) # (4,)
print(c, '\n')

# このような重み付き和の計算は，
print("行列の積による重み付き和\n", a @ hs)
# で行うこともできるが，バッチ処理ができない．  
# その場合は，np.tensordotやnp.einsumを使うことになるが，ここではわかりやすさを優先してrepeatとsumを使う．

hs (5, 4)
[[ 1.65959464 -1.18548218 -1.02005596 -0.313804  ]
 [-0.0666629  -1.62565225  1.31726545 -0.64739142]
 [ 0.67955789 -0.5281331   0.57131534  1.05003435]
 [-0.17295064 -0.1116932  -0.47715419  1.11322819]
 [-0.58930992  0.70504954  1.10845058  1.28380489]] 

a (5,)
[0.8  0.1  0.03 0.05 0.02] 

ar (5, 4)
[[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]] 

t (5, 4)
[[ 1.32767571 -0.94838574 -0.81604477 -0.2510432 ]
 [-0.00666629 -0.16256523  0.13172655 -0.06473914]
 [ 0.02038674 -0.01584399  0.01713946  0.03150103]
 [-0.00864753 -0.00558466 -0.02385771  0.05566141]
 [-0.0117862   0.01410099  0.02216901  0.0256761 ]] 

c (4,)
[ 1.32096243 -1.11827863 -0.66886746 -0.20294381] 

行列の積による重み付き和
 [ 1.32096243 -1.11827863 -0.66886746 -0.20294381]


In [22]:
# バッチ処理版の重み付き和
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
print("hs", hs.shape)

a = np.random.randn(N, T) # 各単語への重み
print("a", a.shape)

ar = a.reshape(N, T, 1).repeat(H, axis=2) # 各単語への重みを隠れベクトルの方向(チャネル方向)へ展開
print("ar", ar.shape)

t = hs * ar
c = np.sum(t, axis=1) # 単語の重み＝各チャネルの重さの総和

print("c", c.shape)
print(c)

hs (10, 5, 4)
a (10, 5)
ar (10, 5, 4)
c (10, 4)
[[-0.68282769  2.45835465 -1.13851056 -3.81966644]
 [ 2.08464124 -0.64927893 -0.97363733 -1.44676433]
 [ 7.50932646  3.83101582  2.97767903 -2.57990066]
 [ 4.18777673  0.36739177 -1.79448791  0.97715596]
 [-5.08643804  0.93466861 -3.48879172  1.91318736]
 [ 1.78318087  2.71911113  2.98107048 -1.52787458]
 [ 1.84168255 -1.80290223 -0.93913331 -0.0993814 ]
 [-2.74574998 -3.71737876  0.6001083  -3.69178694]
 [-1.31499305  1.22049428  1.95301554  3.88925537]
 [-1.39317153  0.03035466 -0.26898013 -2.11163315]]


重み付き和の層をWeightSumクラスとして実装

In [ ]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None
    
    def forward(self, hs, a):
        N, T, H = hs.shape
        
        ar = a.reshape(N, T, 1).repeat(H, axis=2)
        t = hs * ar
        
        c = np.sum(t, axis=1)
        self.cache = (hs, ar)
        
        return c
    
    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape